# Miliardarzy na świecie - dane Forbes API

Dane pochodzą z otwartego API Forbes, udostępniającego informacje o aktualnym [rankingu najbogatszych ludzi na świecie](https://www.forbes.com/real-time-billionaires/) (ale nie zawierają historii zmian majątku).

## 1. Wprowadzenie

Celem notebooka jest pobranie danych o najbogatszych osobach na świecie, bezpośrednio z API Forbes. Następnie zweryfikuję kompletność danych, poprawię ich strukturę oraz zapiszę do pliku CSV w celu dalszej analizy.

> 📌 *Uwaga:* Niniejszy plik zawiera jedynie proces **pozyskiwania i przygotowania danych**. Właściwa analiza z wizualizacjami i wnioskami znajduje się tutaj: [Miliarderzy w liczbach. Kim są najbogatsi ludzie świata?](https://github.com/ElaWajdzik/Ongoing_Projects/blob/main/%2352wykresy2025/03%20-%20miliarderzy%20Forbes.md)

W analizie odpowiadam m.in. na pytania:
* W jakich krajach mieszkają najbogatsi?
* Jak wygląda podział miliartedów według płci?
* Jakie są najpopularniejsze źródła majątku najbogatszych ludzi?
* Jak ma sie bogactwo miliarderów do globalnego bogactwa?

## 2. Pobieranie danych z API Forbes

Dane wystawione są w formacie JSON, bezpośrednio po pobraniu przekszałcę je do formatu Pandas DataFrame.

Lista pobranych informacji:
* `rank` - pozycja na liście na podstawie wartości majątku
* `url` - link do personalnej strony, potrzebny do późniejszej obróbki
* `net_worth` - wartość majątku netto w milionach dolarów obliczony według metodologii Forbes
* `private_asstes_worth` - wartość prywatnego majątku w milionach dolarów
* `full_name` - imię i nazwisko
* `last_name` - nazwisko
* `country` - kraj pochodzenia
* `state` - stan zamieszkania
* `city` - miasto zamieszkania
* `industry` - kategoria branży w której dana osoba działa 
* `source` - źródło bogactwa
* `birth_date` - data urodzenia
* `gender` - płeć (M/F)
* `family_list` - obecnośc na liście rodzinnych fortun (True/False)

In [5]:
# Załodowanie bibliotek
import requests
import json
import pandas as pd

In [6]:
# Pobranie danych z API Forbes
API_URL = "https://www.forbes.com/forbesapi/person/rtb/0/position/true.json?limit=1"
response = requests.get(API_URL)
data = response.json()

# Pobranie pierwszej osoby
first_person = data["personList"]["personsLists"][0]

# Wyświetlenie wszystkich dostępnych kluczy
print("Lista dostępnych kluczy:")
print(first_person.keys())


# Wyświetlenie pełnych danych
# print("Pełne dane:")
# for key, value in first_person.items():
#    print(f"{key}: {value}")

Lista dostępnych kluczy:
dict_keys(['naturalId', 'name', 'year', 'uri', 'rank', 'listUri', 'visible', 'position', 'imageExists', 'bio', 'finalWorth', 'person', 'personName', 'state', 'city', 'source', 'industries', 'countryOfCitizenship', 'timestamp', 'version', 'gender', 'birthDate', 'lastName', 'financialAssets', 'date', 'wealthList', 'estWorthPrev', 'privateAssetsWorth', 'familyList', 'interactive', 'archivedWorth', 'thumbnail', 'squareImage', 'bioSuppress', 'csfDisplayFields', 'bios', 'abouts'])


---
W początkowym etapie pobierania i przekształcania danych pracowałam tylko na pierwszych 5 rekordach zamiast na pełnej liście (ok. 3 tys. rekordów).

In [8]:
# Pobranie tylko 5 pierwszych obserwacji z API Forbes
# API_URL = "https://www.forbes.com/forbesapi/person/rtb/0/position/true.json?limit=5"

# Pobranie danych z API Forbes
API_URL = "https://www.forbes.com/forbesapi/person/rtb/0/position/true.json"

response = requests.get(API_URL)
data = response.json()

In [9]:
# Sprawdzenie liczby osób na liście
num_people = len(data["personList"]["personsLists"])

print(f"Liczba osób na liście: {num_people}")

Liczba osób na liście: 3017


In [10]:
# Przekształcenie na listę
billionaires = []
for person in data["personList"]["personsLists"]:
    billionaires.append({
        "rank": person.get("position", None),
        "url": person.get("uri", None),
        "net_worth":  person.get("finalWorth", 0),
        "private_assets_worth": person.get("privateAssetsWorth", 0),
        "full_name": person.get("personName", None),
        "last_name": person.get("lastName", None),
        "country": person.get("countryOfCitizenship", None),        
        "state": person.get("state", None),
        "city": person.get("city", None),
        "industry": person.get("industries", [None])[0],  # pierwsza pozycja
        "source": person.get("source", None),
        "birth_date": person.get("birthDate", None),
        "gender": person.get("gender", None),
        "family_list": person.get("familyList", None)
    })

# Konwersja do df
df = pd.DataFrame(billionaires)

In [11]:
df.head(5)

,rank,url,net_worth,private_assets_worth,full_name,last_name,country,state,city,industry,source,birth_date,gender,family_list
0,1,elon-musk,424679.497,231260.0,Elon Musk,Musk,United States,Texas,Austin,Automotive,"Tesla, SpaceX",4.691520e+10,M,False
1,2,mark-zuckerberg,221206.205,2437.0,Mark Zuckerberg,Zuckerberg,United States,California,Palo Alto,Technology,Facebook,4.533408e+11,M,False
2,3,jeff-bezos,220874.365,33908.0,Jeff Bezos,Bezos,United States,Florida,Miami,Technology,Amazon,-1.884384e+11,M,False
3,4,larry-ellison,200919.217,14358.0,Larry Ellison,Ellison,United States,California,Woodside,Technology,Oracle,-8.007552e+11,M,False
4,5,warren-buffett,160205.395,1169.0,Warren Buffett,Buffett,United States,Nebraska,Omaha,Finance & Investments,Berkshire Hathaway,-1.241482e+12,M,False


Lista dodatkowych informacje z przekształceń pobranych danych:
* `first_name` - imie wyodrębnione z `full_name`
* `age` - wiek, wyliczony na podstawie `birth_date`
* `birht_year`
* `birth_month`
* `birth_day`

In [13]:
from datetime import datetime

# Konwersja "Birth Date" na datetime
df["birth_date"] = pd.to_datetime(df["birth_date"], unit="ms", errors="coerce")

# Obliczanie wieku
today = datetime.today()
# df["age"] = df["birth_date"].apply(lambda x: today.year - x.year if pd.notnull(x) else None)
df["age"] = df["birth_date"].apply(lambda x: (today.year - x.year - ((today.month, today.day) < (x.month, x.day))) if pd.notnull(x) else None)

# Przekształcenie wieku na int
df["age"] = df["age"].fillna(0).astype(int)

In [14]:
# Dodanie kolumny z rokiem urodzenia, miesiącem i dniem
df["birth_year"] = df["birth_date"].dt.year.fillna(0).astype(int)
df["birth_month"] = df["birth_date"].dt.month.fillna(0).astype(int)
df["birth_day"] = df["birth_date"].dt.day.fillna(0).astype(int)

Na podstawie ogólnie dostępnych danych uzupełniłam brakujące informacje o płci oraz kategori przemysłu.

In [16]:
# Dodanie brakujących informacji o płci
df.loc[df["full_name"].isin(["Viren Doshi", "Brian Hill", "Pantas Sutardja", "Oran Holtzman"]) & (df["gender"].isna()), "gender"] = "M"

# Dodanie brakujących informacji o industry
df.loc[df["full_name"].isin(["Jerry Seinfeld", "Bruce Springsteen"]) & (df["industry"].isna()), "industry"] = "Media & Entertainment"
df.loc[df["full_name"].isin(["Pantas Sutardja", "Yury Maksimov"]) & (df["industry"].isna()), "industry"] = "Technology"
df.loc[df["full_name"].isin(["Brian Hill", "Oran Holtzman"]) & (df["industry"].isna()), "industry"] = "Fashion & Retail"

In [17]:
df.head(3)

,rank,url,net_worth,private_assets_worth,full_name,last_name,country,state,city,industry,source,birth_date,gender,family_list,age,birth_year,birth_month,birth_day
0,1,elon-musk,424679.497,231260.0,Elon Musk,Musk,United States,Texas,Austin,Automotive,"Tesla, SpaceX",1971-06-28,M,False,53,1971,6,28
1,2,mark-zuckerberg,221206.205,2437.0,Mark Zuckerberg,Zuckerberg,United States,California,Palo Alto,Technology,Facebook,1984-05-14,M,False,41,1984,5,14
2,3,jeff-bezos,220874.365,33908.0,Jeff Bezos,Bezos,United States,Florida,Miami,Technology,Amazon,1964-01-12,M,False,61,1964,1,12


In [18]:
# Usuinięce zduplikowanych wierszy
df = df.drop_duplicates()

In [19]:
# Weryfikacja duplikatów
df[df["full_name"].str.contains("Charlene de Carvalho-Heineken", na=False, case=False)]

,rank,url,net_worth,private_assets_worth,full_name,last_name,country,state,city,industry,source,birth_date,gender,family_list,age,birth_year,birth_month,birth_day
160,161,charlene-de-carvalho-heineken,14794.861,4336.233824,Charlene de Carvalho-Heineken & family,de Carvalho-Heineken,Netherlands,None,London,Food & Beverage,Heineken,1954-06-30,F,False,70,1954,6,30


In [20]:
#df.isna().sum()

# Obliczenie procentowych braków danych
missing_percentage = (df.isnull().sum() / len(df)) * 100

print(missing_percentage)

rank                     0.000000
url                      0.000000
net_worth                0.000000
private_assets_worth     0.000000
full_name                0.000000
last_name                0.000000
country                  0.000000
state                   53.994034
city                     1.358966
industry                 0.033146
source                   0.033146
birth_date               1.955585
gender                   0.000000
family_list              0.000000
age                      0.000000
birth_year               0.000000
birth_month              0.000000
birth_day                0.000000
dtype: float64


Najwięcej braków danych (ok 54% wierszy) występuje w kolumnie `state` co będzie powiązane z faktem że nie wszyscy miliarderzy mieszkają na terenie USA. 

Dodatkowo dla 1% wierszy pojawiają się braki danych w kolumnie `city` i `birth_date`. 


Ponieważ dane są w trybie realtime, całość zapisuję do pliku CSV, aby umożliwić dalszą analizę lub powrót do nich w przyszłości.

In [23]:
# Zapisanie do pliku CSV
df.to_csv("forbes_billionaires_lista.csv", index=False)

---

## 3. Pobranie dodatkowych danych z [real-time billionairs API](https://realtimebillionaires.de/api)

[Opis działania real-time billionaires API](https://github.com/komed3/rtb-api/tree/main)

Z pośrednictwem tego API dodatkowo pobrałam dane o:
* `url` - klucz wartości do połączenia danych między plikami
* `self_made` - flaga oznaczająca czy majątek jest dziedziczony czy samodzielnie wypracowany (True - self-made, False - odziedziczony)
* `self_made_type` - kategoria typu majątku: self-made, inherited and growing i inherited
* `education_school` - nazwa szkoły lub uczelni, w której uzyskano wykształcenie
* `education_degree` - stopień wykształcenia 

link pomocniczy https://github.com/komed3/rtb-api/tree/main

In [26]:
import time
import concurrent.futures

#people = df["url"].tolist()  # pełna lista osób
people = [
    "elon-musk", "mark-zuckerberg", "jeff-bezos", "larry-ellison", "bernard-arnault",
    "warren-buffett", "larry-page", "sergey-brin", "amancio-ortega", "wang-yunan",
    "william-chisholm", "mehmet-ali-aydinlar"
]

base_url = "https://cdn.jsdelivr.net/gh/komed3/rtb-api@main/api/profile/{}/info"

def fetch_data(person):
    url = base_url.format(person)
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        education_list = data.get("education", [])
        education_school = education_list[0].get("school", "") if education_list else ""
        education_degree = education_list[0].get("degree", "") if education_list else ""

        return {
            "url": data.get("uri", ""),
            "self_made": data.get("selfMade", {}).get("_is", ""),
            "self_made_type": data.get("selfMade", {}).get("type", ""),
            "source": ", ".join(data.get("source", [])) if isinstance(data.get("source", []), list) else data.get("source", ""),
            "education_school": education_school,
            "education_degree": education_degree
        }
    elif response.status_code == 429:  # Too Many Requests
        print("🔴 API zwróciło 429 - czekam 10s...")
        time.sleep(10)  # Czekaj 10 sekund i spróbuj ponownie
        return fetch_data(person)
    else:
        print(f"Błąd {response.status_code} dla {person}")
        return None

# Pobieranie w partiach (bez przeciążania API)
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    results = list(executor.map(fetch_data, people))

processed_data = [r for r in results if r is not None]
df_extended = pd.DataFrame(processed_data)

df_extended.head()

,url,self_made,self_made_type,source,education_school,education_degree
0,elon-musk,True,self-made,"Tesla, SpaceX",University of Pennsylvania,Bachelor of Arts/Science
1,mark-zuckerberg,True,self-made,Facebook,Harvard University,Drop Out
2,jeff-bezos,True,self-made,Amazon,Princeton University,Bachelor of Arts/Science
3,larry-ellison,True,self-made,Oracle,University of Chicago,Drop Out
4,bernard-arnault,False,inherited and growing,LVMH,Ecole Polytechnique de Paris,Bachelor of Arts/Science


In [27]:
#df_extended.isna().sum()

# Obliczenie procentowych braków danych
missing_percentage_extended = (df_extended.isnull().sum() / len(df_extended)) * 100

print(missing_percentage_extended)

url                 0.0
self_made           0.0
self_made_type      0.0
source              0.0
education_school    0.0
education_degree    0.0
dtype: float64


In [28]:
# Zapisanie do pliku CSV
df_extended.to_csv("forbes_billionaires_extended.csv", index=False)